In [1]:
import re

import altair as alt
from altair import datum
import numpy as np
import pandas as pd


# Get Data
## Extract Citation Numbers from Bibtex file

In [2]:
p2bbl_file = r'main.bbl' # path to .bbl-file
with open(p2bbl_file, 'r') as fp:
    fContent = fp.read()
    
pattern = r'\\bibitem\[.*?\]\{(.+?)\}' # use raw string format 'r' -> see https://docs.python.org/3/howto/regex.html#the-backslash-plague
re.compile(pattern)
matchObj_list = re.findall(pattern, fContent, flags=re.DOTALL) # re.DOTALL is important because of newlines within brackets 
df_cit = pd.DataFrame({'Reference: Abbreviation': matchObj_list, 'Reference: Number': range(1,len(matchObj_list)+1)})
df_cit['Reference: Number'] = '[' + df_cit['Reference: Number'].astype(str) + ']'
df_cit['Reference: Abbreviation'] = df_cit['Reference: Abbreviation'].astype(str) 
# df_cit.head()

In [3]:
## Read Performance Values

In [4]:
path = r'DNN-NILM_low-freq_Performance.xlsx'
df = pd.read_excel(path)
unnamed_columns = []
for column in df.columns:
    if column.find('Unnamed:') != -1:
        unnamed_columns.append(column)
df = df.drop(columns=unnamed_columns)
df['Reference: Abbreviation'] = df['Reference: Abbreviation'].astype(str) 
# df.head()

In [5]:
# Sanitize data
# replace '?' with -1 values in case of 'Input: Window Size'
df.loc[ df['Input: Window Size'] == '?', 'Input: Window Size'] = -1
df.loc[ df['Input: Window Size'] == 'nan', 'Input: Window Size' ] = -1
df.loc[ df['Input: Window Size'] == 'not appl', 'Input: Window Size' ] = -1
df.loc[ df['Input: Window Size'] == np.nan, 'Input: Window Size'] = -1
df['Input: Window Size'] = df['Input: Window Size'].astype(float)

In [6]:
# Join dataframes with citation keys
df = df.merge(df_cit, on='Reference: Abbreviation', how='left')
df = df.rename(columns={'Reference: Number_y': 'Reference: Number'})

## Evaluation on Metrics for noised, unseen
(Done directly in Excel):
* classificaton
    * F1: 12 publications
* regression
    * MAE: 20 publications
    * SAE:  12 publications
    * EstAcc: 7 publications

# Investigate: F1 for noised, unseen case

In [7]:
df_f1 = df[ (df['Performance Metric: Type'] == 'F1') & 
            (df['Training: Type of Data'] == 'noised') & 
            (df['Evaluation: Scenario'] == 'unseen')].copy()
df_f1 = df_f1[['Reference: Abbreviation', 'Reference: Number', 
                'Appliance', 'Training: Dataset', 'Model: Basic Type', 'Model: Denomination', 
                'Input: Window Size', 'Input: Sampling Rate', 'Performance Metric: Value']].copy()

In [8]:
print(df_f1['Reference: Abbreviation'].unique())
print(len(df_f1['Reference: Abbreviation'].unique()))

['kelly2015' 'bonfigli2018' 'barsim2018' 'nascimento2016' 'murray2019'
 'krystalakos2018' 'sudoso2019' 'linh2019' 'cavdar2019' 'yue2020'
 'massidda2020' 'rafiq2020' 'kukunuri2020']
13


In [9]:
# Remove Baseline Model 
df_f1 = df_f1[ ~(df_f1['Model: Denomination'] == 'BL =0') ]
# Note to self: Ke15, Zh2016 are baseline/reference models -> remove these Models from other authors
df_f1 = df_f1[ ~(df_f1['Model: Denomination'].str.contains('Ke15') | 
                 df_f1['Model: Denomination'].str.contains('Zh2016') | 
                 df_f1['Model: Denomination'].str.contains('He2016')) ] 

In [10]:
# Group Results
grouped = df_f1.groupby(['Reference: Abbreviation', 'Appliance', 'Training: Dataset'])
list_of_maxs = []
for group in grouped:
    group = group[1]
    min_row = group[ group['Performance Metric: Value'] == group['Performance Metric: Value'].max()]
    list_of_maxs.append(min_row)
df_f1 = pd.concat(list_of_maxs)
print(df_f1['Model: Denomination'].unique())
print(df_f1['Appliance'].unique())
print(df_f1['Training: Dataset'].unique())

['CNN dAE' 'CNN-RNN' 'LSTM-bi' 'dAE' 'CNN RctAng' 'GRU-bi' 'CNN s2p'
 'STL Iterative Pruning (30%)' 'STL Rank 4 Tensor D.' 'RNN' 'TP-NILM'
 'RNN-GRU' 'CNN' 'RecCNN' 'MFS-LSTM' 'S2SwA' 'BERT']
['dishwasher' 'fridge' 'kettle' 'microwave' 'washing machine'
 'tumble dryer']
['UK-DALE' 'REDD' 'REFIT']


In [11]:
# Create column that combines 'Reference' and 'Number' to check that numbers are correct
df_f1['No-Ref'] = df_f1['Reference: Abbreviation'] + '_' + df_f1['Reference: Number']

In [12]:
datasets = df_f1['Training: Dataset'].unique()
for dataset in datasets:
    print(dataset, ': ', np.sum(df_f1['Training: Dataset']==dataset))

UK-DALE :  39
REDD :  24
REFIT :  5


In [13]:
appliances = df_f1['Appliance'].unique()
for app in appliances:
    print(app, ': ', np.sum(df_f1['Appliance'] == app))

dishwasher :  17
fridge :  17
kettle :  8
microwave :  14
washing machine :  11
tumble dryer :  1


In [14]:
# remove appliances with low number of results
df_f1 = df_f1[~(df_f1['Appliance'] == 'tumble dryer')]
# # df_f1 = df_f1[~((df_f1['Appliance'] == 'lighting'))]
# df_f1 = df_f1[~((df_f1['Appliance'] == 'cooker'))]
# df_f1 = df_f1[~((df_f1['Appliance'] == 'shower'))]
# df_f1 = df_f1[~((df_f1['Appliance'] == 'stove/oven'))]

# remove Morgan2017 -> works on proprietary dataset & only fridge
# df_f1 = df_f1[~(df_f1['Reference: Abbreviation'] == 'Morgan2017') ]

In [15]:
# Average number of results per appliance
appliances = df_f1['Appliance'].unique()
n_apps = 0
for app in appliances:
    n_apps = n_apps + np.sum(df_f1['Appliance'] == app)
print(n_apps/len(appliances)/4)

3.35


In [16]:
# find 'na' Reference: Numbers
# df['Reference: Number'].isna()
df['Reference: Abbreviation'][df['Reference: Number'].isna()].unique()

array([], dtype=object)

## Overview 

In [17]:
# Visualization ,
upper_bound = 1 #14
sortByDataset = alt.Chart().mark_point(clip=True, size=90, filled=True).encode(
    x=alt.X('Training: Dataset:N', title='Dataset'),
    y=alt.Y('Performance Metric: Value:Q', title='F1-score', scale=alt.Scale(domain=(0, upper_bound))),
#    color=alt.Color('Reference: Abbreviation:N', legend=alt.Legend(title='Publication'), scale=alt.Scale(scheme = 'category20')),
    color=alt.Color('Reference: Number:N', legend=alt.Legend(title='Publication'), scale=alt.Scale(scheme = 'category20')),
#    color=alt.Color('Input: Sampling Rate:Q', legend=alt.Legend(title='Sampling Rate'), scale=alt.Scale(domain=(0,10), type='symlog')),
#    color=alt.Color('Input: Window Size:Q', legend=alt.Legend(title='Window Size'), scale=alt.Scale(type='log')),
).transform_filter(
    alt.FieldOneOfPredicate(field='Training: Dataset', oneOf=['REDD', 'UK-DALE', 'REFIT'])
)
# text = alt.Chart().mark_text(
#     clip=True, align='left', dy=-5, dx=-24,
#     fontSize=13
# ).encode(
#     x=alt.X('Training: Dataset:N'),
#     y=alt.Y('Performance Metric: Value:Q', title='MAE [W]', scale=alt.Scale(domain=(0, upper_bound))),
#     text = 'Reference: Abbreviation:N'
# )

alt.layer(
    sortByDataset,  
    data=df_f1, width=90#, height=500
).facet(
    column=alt.Column('Appliance:N', title=None)
).configure_facet(
    spacing=7
).configure_header(
    titleFontSize=14,
    labelFontSize=14
)

alt.FacetChart(...)

## Variant ZOOM

In [18]:
# Individualized focus views
b = 20 
h = 250
params = {
    'Kettle':          { 'app' : 'kettle',          'width': b*1, 'height': h, 'upper_bound': 1, 'lower_bound': 0.85 }, 
    'Microwave':       { 'app' : 'microwave',       'width': b*3, 'height': h, 'upper_bound': 1, 'lower_bound': 0.85 }, 
    'Dishwasher':      { 'app' : 'dishwasher',      'width': b*3, 'height': h, 'upper_bound': 1, 'lower_bound': 0.7 },
    'Washing machine': { 'app' : 'washing machine', 'width': b*3, 'height': h, 'upper_bound': 1, 'lower_bound': 0.7 },
    'Fridge':          { 'app' : 'fridge',          'width': b*3, 'height': h, 'upper_bound': 1, 'lower_bound': 0.7 }, 
}

In [19]:
# Visualization Dishwasher,
layers = []
for key, par in params.items():
    upper_bound = par['upper_bound']
    lower_bound = par['lower_bound']
    width = par['width']
    height = par['height']
    app = par['app']
    sortByDataset = alt.Chart().mark_point(clip=True, size=90, filled=True).encode(
        x=alt.X('Training: Dataset:N', title='Dataset'),
        y=alt.Y('Performance Metric: Value:Q', title='F₁', scale=alt.Scale(domain=(lower_bound, upper_bound))),
        color=alt.Color('Reference: Number:N', legend=alt.Legend(title='Publication'), scale=alt.Scale(scheme = 'category20')), 
#       color=alt.Color('No-Ref:N', legend=alt.Legend(title='Publication'), scale=alt.Scale(scheme = 'category20')), 
#        color=alt.Color('Reference: Abbreviation:N', legend=alt.Legend(title='Publication'), scale=alt.Scale(scheme = 'category20')), 
#        shape=alt.Shape('Model: Basic Type', sort=['recurrent', 'feedforward'], legend=alt.Legend(title='Network Type'))
#        shape=alt.Shape('Model: Basic Type', sort=['recurrent', 'feedforward'], legend=alt.Legend(title='Network Type'))
    ).transform_filter(
        alt.FieldOneOfPredicate(field='Appliance', oneOf=[app])
    ).properties(
        title=key
    )
#    text = alt.Chart().mark_text(
#        clip=True, align='left', dy=-7, dx=-15,
#        fontSize=13
#    ).encode(
#        x=alt.X('Training: Dataset:N'),
#        y=alt.Y('Performance Metric: Value:Q', title='MAE [W]', scale=alt.Scale(domain=(lower_bound, upper_bound))),
#    #    text = 'Reference: Abbreviation:N'
#        text = 'Reference: Number:N'
#    ).transform_filter(
#        alt.FieldOneOfPredicate(field='Appliance', oneOf=[key])
#    )
    layers.append(alt.layer(
        sortByDataset, 
        data=df_f1, width=width, height=height
    ))
alt.hconcat(*layers, spacing=50, bounds='flush')

alt.HConcatChart(...)

Note to self: I double checked the values for the top result in the corresponding publications. *They are correct.*

In [20]:
# VERSION FOR PUBLICATION
# Visualization ,
upper_bound = 1 #14
lower_bound = 0.7
sortByDataset = alt.Chart().mark_point(clip=True, size=90, filled=True).encode(
    x=alt.X('Training: Dataset:N', title='Dataset'),
    y=alt.Y('Performance Metric: Value:Q', title='F1-score', scale=alt.Scale(domain=(lower_bound, upper_bound))),
#    color=alt.Color('Reference: Abbreviation:N', legend=alt.Legend(title='Publication'), scale=alt.Scale(scheme = 'category20')),
    color=alt.Color('Reference: Number:N', legend=alt.Legend(title='Publication'), scale=alt.Scale(scheme = 'category20')),
)

alt.layer(
    sortByDataset,  
    data=df_f1, width=90#, height=500
).facet(
    column=alt.Column('Appliance:N', title=None)
).configure_facet(
    spacing=7
).configure_header(
    titleFontSize=14,
    labelFontSize=14
)

alt.FacetChart(...)

In [21]:
# Evaluate x best performing Models
apps = ['kettle', 'microwave', 'dishwasher', 'washing machine', 'fridge']
authors = []
for app in apps:
    df_app = df_f1[ df_f1['Appliance'] == app]
    df_app = df_app.sort_values('Performance Metric: Value')
    first_i_values = round(len(df_app)/4)
    df_app = df_app.iloc[-first_i_values:,:]
    print(app)
    print(first_i_values)
    authors.extend(df_app['Reference: Abbreviation'].values)
    print(df_app)
#    print('publications: ', df_app['Reference: Abbreviation'].values)
#    print('values', df_app['Performance Metric: Value'].values)
#    print('datasets: ', df_app['Training: Dataset'].unique())
#    print('no of ff: ', np.sum(df_app['Model: Basic Type'] == 'feedforward'), 
#          'no of rnn: ',  np.sum(df_app['Model: Basic Type'] == 'recurrent'), 
#          'no of combined', np.sum(df_app['Model: Basic Type'] == 'combined') )
    print()

kettle
2
     Reference: Abbreviation Reference: Number Appliance Training: Dataset  \
1497               rafiq2020              [61]    kettle           UK-DALE   
1048              sudoso2019             [100]    kettle           UK-DALE   

     Model: Basic Type Model: Denomination  Input: Window Size  \
1497          combined            MFS-LSTM                -1.0   
1048          combined               S2SwA               768.0   

     Input: Sampling Rate  Performance Metric: Value            No-Ref  
1497                    1                      0.965    rafiq2020_[61]  
1048                    6                      0.967  sudoso2019_[100]  

microwave
4
     Reference: Abbreviation Reference: Number  Appliance Training: Dataset  \
584               murray2019              [55]  microwave              REDD   
587               murray2019              [55]  microwave              REDD   
498           nascimento2016             [111]  microwave              REDD   
1199     

In [22]:
authors = np.asarray(authors)
counts = []
for author in pd.Series(authors).unique():
    count = np.sum(authors == author)
    counts.append(count)
counts = pd.DataFrame({'authors': pd.Series(authors).unique(), 'counts': counts})
counts.sort_values('counts', ascending=False)

,authors,counts
2,murray2019,6
3,nascimento2016,3
0,rafiq2020,2
4,cavdar2019,2
5,massidda2020,2
1,sudoso2019,1
6,barsim2018,1
